In [1]:
from tqdm.contrib.concurrent import process_map

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.decode import print_breakdown, search

# TODO: handle non-US cars
df_nhtsa = await search(
  include_openpilot=True,
  include_police=True,
  skip_missing_asbuilt=True,
)

# pre-load asbuilt
process_map(AsBuiltData.from_vin, df_nhtsa['VIN'].unique(), desc='Loading AsBuilt Data', chunksize=100)

print()
print_breakdown(df_nhtsa, include_model_year=False)

Loaded 36766 VINs (filter_comment=None, include_openpilot=True, skipped=162, missing_asbuilt=0)


Loading AsBuilt Data:   0%|          | 0/36766 [00:00<?, ?it/s]


Model
                      2
Aviator            1107
Bronco              997
Bronco Sport       1500
C-Max                 5
Continental          23
Corsair             986
E-Transit           157
Ecosport            406
Edge               2685
Escape             3569
Expedition          835
Expedition MAX      733
Explorer           3364
F-150              8254
F-150 Lightning     385
F-250              1720
F-350              1158
F-450               207
F-550                 1
Fiesta              196
Flex                189
Focus               169
Fusion             1163
GT                    3
MKC                  55
MKT                   7
MKZ                 100
Maverick            994
Mustang             952
Mustang Mach-E      839
Nautilus           1169
Navigator           303
Navigator L         230
Ranger              533
Taurus              122
Transit            1234
Transit Connect     414
dtype: int64


In [2]:
df = df_nhtsa[['VIN', 'Make', 'Model', 'ModelYear']].copy()
df['CarName'] = df['Make'] + ' ' + df['Model'] + ' ' + df['ModelYear'].astype(str)
df.drop(columns=['Make', 'Model', 'ModelYear'], inplace=True)
df.head()

,VIN,CarName
0,1FTFW1E80PFB96171,FORD F-150 2023
1,1FMEE5DP4PLB92829,FORD Bronco 2023
2,3LN6L5FCXKR613421,LINCOLN MKZ 2019
3,1FT8W3DTXMED39655,FORD F-350 2021
4,1FTER4FH3KLA17604,FORD Ranger 2019


In [3]:
import pandas as pd

from panda.python.uds import DATA_IDENTIFIER_TYPE
from notebooks.ford.ecu import FordEcu

ecu_map = {
  'ABS': FordEcu.AntiLockBrakeSystem,
  'APIM': FordEcu.AccessoryProtocolInterfaceModule,
  'IPMA': FordEcu.ImageProcessingModuleA,
  'PSCM': FordEcu.PowerSteeringControlModule,
}

def get_ecu_platform_code(abd: AsBuiltData, ecu: FordEcu) -> str | None:
  fw = abd.get_identifier(ecu, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER)
  if fw is None:
    return None
  prefix, core, _ = fw.split('-')
  # core doesn't help split any platforms apart
  # the first year of the prefix (somewhat related to the year) does help split model generations
  return prefix
  # return f'{core}-{prefix}'

def get_platform_codes(row) -> pd.Series:
  abd = AsBuiltData.from_vin(row['VIN'])
  values: dict[str, str | None] = {}
  for ecu_name, ecu in ecu_map.items():
    values[ecu_name] = get_ecu_platform_code(abd, ecu)
  return pd.Series(values)

df_fw = df.join(df.apply(get_platform_codes, axis=1)).drop(columns=['VIN']).drop_duplicates()
df_fw.head()

,CarName,ABS,APIM,IPMA,PSCM
0,FORD F-150 2023,PL34,PU5T,PJ6T,ML3V
1,FORD Bronco 2023,MB3C,NU5T,M2DT,NB3C
2,LINCOLN MKZ 2019,KG9C,1U5T,HS7T,KG9C
3,FORD F-350 2021,LC3C,1U5T,None,None
4,FORD Ranger 2019,KB3C,1U5T,KB3T,JR3C


In [4]:
from notebooks.utils.union import merge_sets


def group_by(df: pd.DataFrame, by: list[str]) -> None:
  car_groups = list()
  for group in df.groupby(by, dropna=False):
    cars = set(group[1]['CarName'].unique())
    car_groups.append(cars)

  return merge_sets(car_groups)

In [5]:
from notebooks.ford.platforms import find_openpilot_platform

for ecus in (
  # ('ABS', 'PSCM'), - combines escape and bronco sport
  # ('ABS', 'APIM'), - combines escape and bronco sport
  # ('IPMA', 'PSCM'),
  # ('IPMA', 'APIM'),
  # ('PSCM', 'IPMA', 'APIM'), - combines escape and bronco sport
  # ('ABS', 'PSCM', 'APIM'),  - combines escape and bronco sport
  # ('ABS', 'IPMA', 'APIM'),
  ('ABS', ),
  ('ABS', 'IPMA'),  # seperate Bronco Sport and Escape
  ('ABS', 'IPMA', 'PSCM'),  # splits out: 2019 Mustang (from 2020-23), 2024 Bronco (from 2021-23)
  ('ABS', 'IPMA', 'PSCM', 'APIM'),  # splits out: 2019 EcoSport (from 2020-22), 2024 Escape/Corsair (from 2023)
):
  car_groups = group_by(df_fw, list(ecus))
  print(', '.join(ecus))
  print(f'Found {len(car_groups)} distinct car groups')
  for group in car_groups:
    # platforms = {find_openpilot_platform(car) for car in group}
    # print(f'{len(platforms)} platforms: {", ".join(map(str, platforms)):<33}; {", ".join(group)}')
    print('\t' + ', '.join(group))
  print()

ABS
Found 50 distinct car groups
	FORD Escape 2015
	FORD Fiesta 2017, FORD Fiesta 2018, FORD Fiesta 2019
	FORD Focus 2017, FORD Focus 2018
	FORD Edge 2018
	FORD Flex 2019, FORD Taurus 2019, LINCOLN MKT 2019, FORD Taurus 2018, FORD Flex 2018
	FORD Transit 2018, FORD Transit 2019
	LINCOLN Continental 2020, LINCOLN Continental 2019
	FORD F-150 2016
	FORD Ecosport 2021, FORD Ecosport 2018, FORD Ecosport 2022, FORD Ecosport 2019, FORD Ecosport 2020
	FORD Mustang 2018
	FORD Transit Connect 2018, FORD Escape 2018, FORD Escape 2019
	FORD C-Max 2018
	FORD Explorer 2017, FORD Explorer 2019, FORD Explorer 2018
	FORD F-450 2018, FORD F-250 2018, FORD F-350 2018
	FORD GT 2020
	FORD Fusion 2018
	FORD F-150 2017, FORD F-150 2018
	LINCOLN  2019, FORD Expedition MAX 2019, FORD Expedition MAX 2018, LINCOLN Navigator 2019, LINCOLN Navigator L 2019, FORD Expedition 2018, FORD Expedition 2019
	FORD Edge 2019, LINCOLN Nautilus 2019
	FORD Ranger 2023, FORD Ranger 2020, FORD Ranger 2022, FORD Ranger 2021, FOR

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /mnt/c/Users/camer/AppData/Local/Programs/Microsoft VS Code


In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

df_reduced_ecus = df_fw.groupby(by=['ABS', 'IPMA'])[['CarName']].agg(set)
df_reduced_ecus['CarInfoPlatforms'] = df_reduced_ecus['CarName'].apply(lambda x: {find_openpilot_platform(car_name) for car_name in x})
df_reduced_ecus

CarName  \
ABS  IPMA                                                                                                                                                                                                                       
F1FC F1FT                                                                                                                                                                                                   {FORD Focus 2018}   
F2GC FL3T                                                                                                                                                                                                    {FORD Edge 2018}   
FG13 DA5T                                                                                                                                                              {FORD Taurus 2019, LINCOLN MKT 2019, FORD Taurus 2018}   
     EG1T                                                                                                                                                              {FORD Taurus 2019, LINCOLN MKT 2019, FORD Taurus 2018}   
FK41 CK4T                                                                                                                                                                              {FORD Transit 2018, FORD Transit 2019}   
G3GC GD9T                                                                                                                                                                {LINCOLN Continental 2020, LINCOLN Continental 2019}   
GV61 F1FT                                                                                                                                                                                {FORD Escape 2018, FORD Escape 2019}   
HB53 GB5T                                                                                                                                                                            {FORD Explorer 2019, FORD Explorer 2018}   
HC3C HC3T                                                                                                                                                                 {FORD F-450 2018, FORD F-250 2018, FORD F-350 2018}   
HG9C HS7T                                                                                                                                                                                                  {FORD Fusion 2018}   
HL34 HL3T                                                                                                                                                                                                   {FORD F-150 2018}   
JL14 JL1T                                                                  {LINCOLN Navigator L 2019, FORD Expedition 2018, FORD Expedition 2019, FORD Expedition MAX 2019, FORD Expedition MAX 2018, LINCOLN Navigator 2019}   
JL34 JL3T                                                                                                                                                                                                   {FORD F-150 2018}   
JR3C JR3T                                                                                                                                                                                                 {FORD Mustang 2018}   
K2GC KA1T                                                                                                                                                                                             {LINCOLN Nautilus 2019}   
     KT4T                                                                                                                                                                                                    {FORD Edge 2019}   
KB3C KB3T                                                                                                                          {FORD Ranger 2020, FORD Ranger 20

In [7]:
from collections import defaultdict
from typing import TypeVar

K, V = TypeVar('K'), TypeVar('V')

def default_dict_to_dict(d: defaultdict[K, V]) -> dict[K, V]:
  return {k: default_dict_to_dict(v) if isinstance(v, dict) else v for k, v in d.items()}

In [8]:
platforms_by_ecu_fw = defaultdict(lambda: defaultdict(set))

index_names = df_reduced_ecus.index.names
for index, row in df_reduced_ecus.iterrows():
  for platform in row['CarInfoPlatforms']:
    if platform is None:
      continue
    for ecu_name, ecu_fw in zip(index_names, index, strict=True):
      platforms_by_ecu_fw[ecu_name][ecu_fw].add(platform)

platforms_by_ecu_fw: dict[str, dict[str, set[str]]] = default_dict_to_dict(platforms_by_ecu_fw)
platforms_by_ecu_fw

{'ABS': {'HC3C': {<CAR.SUPER_DUTY_MK4: 'FORD SUPER DUTY 4TH GEN'>},
  'HL34': {<CAR.F_150_MK13: 'FORD F-150 13TH GEN'>},
  'JL34': {<CAR.F_150_MK13: 'FORD F-150 13TH GEN'>},
  'K2GC': {<CAR.EDGE_MK2_5: 'FORD EDGE 2ND GEN FACELIFT'>},
  'KC3C': {<CAR.SUPER_DUTY_MK4: 'FORD SUPER DUTY 4TH GEN'>},
  'KL34': {<CAR.F_150_MK13: 'FORD F-150 13TH GEN'>},
  'KL3V': {<CAR.F_150_MK13: 'FORD F-150 13TH GEN'>},
  'KR3C': {<CAR.MUSTANG_MK6_5: 'FORD MUSTANG 6TH GEN FACELIFT'>},
  'KV6C': {<CAR.TRANSIT_CONNECT_MK2_5: 'FORD TRANSIT CONNECT 2ND GEN FACELIFT'>,
   <CAR.TRANSIT_CONNECT_MK3: 'FORD TRANSIT CONNECT 3RD GEN'>},
  'L1MC': {<CAR.EXPLORER_MK6: 'FORD EXPLORER 6TH GEN'>},
  'L2GC': {<CAR.EDGE_MK2_5: 'FORD EDGE 2ND GEN FACELIFT'>},
  'LC3C': {<CAR.SUPER_DUTY_MK4: 'FORD SUPER DUTY 4TH GEN'>},
  'LJ9C': {<CAR.MUSTANG_MACH_E_MK1: 'FORD MUSTANG MACH-E 1ST GEN'>},
  'LK41': {<CAR.TRANSIT_MK4_5: 'FORD TRANSIT 4TH GEN FACELIFT'>},
  'LK9C': {<CAR.MUSTANG_MACH_E_MK1: 'FORD MUSTANG MACH-E 1ST GEN'>},
  'LR3C

In [13]:
for ecu_name, platforms_by_fw in platforms_by_ecu_fw.items():
  for fw, platforms in platforms_by_fw.items():
    if not platforms:
      continue
    print(f'{ecu_name:<6} prefix={fw}   {", ".join(platforms)}')

ABS    prefix=HC3C   FORD SUPER DUTY 4TH GEN
ABS    prefix=HL34   FORD F-150 13TH GEN
ABS    prefix=JL34   FORD F-150 13TH GEN
ABS    prefix=K2GC   FORD EDGE 2ND GEN FACELIFT
ABS    prefix=KC3C   FORD SUPER DUTY 4TH GEN
ABS    prefix=KL34   FORD F-150 13TH GEN
ABS    prefix=KL3V   FORD F-150 13TH GEN
ABS    prefix=KR3C   FORD MUSTANG 6TH GEN FACELIFT
ABS    prefix=KV6C   FORD TRANSIT CONNECT 3RD GEN, FORD TRANSIT CONNECT 2ND GEN FACELIFT
ABS    prefix=L1MC   FORD EXPLORER 6TH GEN
ABS    prefix=L2GC   FORD EDGE 2ND GEN FACELIFT
ABS    prefix=LC3C   FORD SUPER DUTY 4TH GEN
ABS    prefix=LJ9C   FORD MUSTANG MACH-E 1ST GEN
ABS    prefix=LK41   FORD TRANSIT 4TH GEN FACELIFT
ABS    prefix=LK9C   FORD MUSTANG MACH-E 1ST GEN
ABS    prefix=LR3C   FORD MUSTANG 6TH GEN FACELIFT
ABS    prefix=LX6C   FORD BRONCO SPORT 1ST GEN, FORD ESCAPE 4TH GEN
ABS    prefix=M2GC   FORD EDGE 2ND GEN FACELIFT
ABS    prefix=MK41   FORD TRANSIT 4TH GEN FACELIFT
ABS    prefix=ML34   FORD F-150 14TH GEN
ABS    prefix=

In [15]:
def get_fw_prefix(fw: str) -> str:
  return fw.split('-')[0]

def try_fingerprint(row) -> set[str]:
  abs_fw, ipma_fw = row[['ABS', 'IPMA']]
  if abs_fw is None:
    return set()
  abs_fw_prefix = get_fw_prefix(abs_fw)
  abs_platforms = platforms_by_ecu_fw['ABS'].get(abs_fw_prefix, set())
  if len(abs_platforms) <= 1:
    return abs_platforms
  print(f'Multiple platforms after ABS: {abs_fw_prefix}: {abs_platforms}')

  if ipma_fw is None:
    return abs_platforms
  ipma_fw_prefix = get_fw_prefix(ipma_fw)
  ipma_platforms = platforms_by_ecu_fw['IPMA'].get(ipma_fw_prefix, set())

  platforms = abs_platforms & ipma_platforms
  if len(platforms) > 1:
    print(f'Multiple platforms after IPMA: {ipma_fw_prefix}: {ipma_platforms} (combined: {platforms})')
  return platforms


df_sample = df_fw[['CarName', 'ABS', 'IPMA']].sample(n=40).copy()
df_sample['Fingerprint'] = df_sample.apply(try_fingerprint, axis=1)
df_sample

Multiple platforms after ABS: LX6C: {<CAR.BRONCO_SPORT_MK1: 'FORD BRONCO SPORT 1ST GEN'>, <CAR.ESCAPE_MK4: 'FORD ESCAPE 4TH GEN'>}


,CarName,ABS,IPMA,Fingerprint
362,FORD F-150 2021,ML34,ML3T,{FORD F-150 14TH GEN}
28619,FORD F-150 2020,None,KL3T,{}
3147,FORD F-150 2021,PL3V,ML3T,{FORD F-150 14TH GEN}
2535,FORD F-350 2023,PC3C,PC3T,{FORD SUPER DUTY 5TH GEN}
348,FORD F-350 2019,KC3C,None,{FORD SUPER DUTY 4TH GEN}
15276,FORD Mustang 2020,LR3C,JR3T,{FORD MUSTANG 6TH GEN FACELIFT}
26,FORD Bronco Sport 2024,LX6C,M1PT,{FORD BRONCO SPORT 1ST GEN}
2015,FORD Expedition MAX 2022,NL14,ML3T,{FORD EXPEDITION 4TH GEN FACELIFT}
42,FORD Edge 2024,P2GC,KT4T,{FORD EDGE 2ND GEN FACELIFT}
730,LINCOLN Navigator 2023,PL14,ML3T,{FORD EXPEDITION 4TH GEN FACELIFT}
